## Move to python file

In [ ]:
def create_resolved_ibcontract(stock_to_trade):
    # Create the contract object
    ibcontract = ib.IBcontract()
    ibcontract.secType = "STK"
    ibcontract.symbol = stock_to_trade
    ibcontract.currency = "USD"
    resolved_ibcontract = app.resolve_ib_contract(ibcontract)
    print(resolved_ibcontract)

    # TODO: Investigate that we are getting the right stock out of all of the options
    return resolved_ibcontract

In [ ]:
def get_latest_price(resolved_ibcontract):
    
    # Get the last price
    tickerid = app.start_getting_IB_market_data(resolved_ibcontract)
    time.sleep(1)
    market_data1 = app.stop_getting_IB_market_data(tickerid)
    market_data1_as_df = market_data1.as_pdDataFrame()
    display(market_data1_as_df.resample("1S").last())
    some_quotes = market_data1_as_df.resample("1S").last()[["last_trade_price"]]
    current_price = some_quotes.iloc[0][0]
    print(current_price)
    
    return current_price
    # TODO: Skip this stock if price comes back as nan

## Reading my Chinese Alipay file. Not related to algo

In [ ]:
from chardet import detect
encoding = lambda x: detect(x)['encoding']

In [ ]:
f = open('/Users/chingaling/Desktop/alipay_record_20181112_1301_1.csv','r')

In [ ]:
type(f)

In [ ]:
for line in f:
    print encoding(line)
f.close()

In [ ]:
  n_line=unicode(line,encoding(line),errors='ignore')
  print n_line
  print n_line.encode('utf8')

## Beginning of algo playground

In [3]:
import sys
sys.path.insert(0, '/Users/chingaling/Documents/Algo Trading/BroCodePC')
import IB_API_tutorial as ib
import main as m
import time
import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
import math
import datetime
from itertools import cycle

In [143]:
app.disconnect()
from importlib import reload
reload (ib)

<module 'IB_API_tutorial' from '/Users/chingaling/Documents/Algo Trading/BroCodePC/IB_API_tutorial.py'>

In [ ]:
sys.path.insert(0, '/Users/chingaling/Documents/Algo Trading/RobCarver')
import L3StreamData

## Create some global variables and open connection

In [5]:
port = 7496
ip = "127.0.0.1"
clientID = 1
buyFactor = .99
SellFactor = 1.01
rebalance_interval = 10
maxBuyOrdersAtOnce = 30
minInvSize = 1000
stock_list = []
maxCandidates=100
leastPrice=3.00
mostPrice=20.00
fireSalePrice = leastPrice
fireSaleAge = 6

# Local file addresses
csvAddress_stocks_to_trade = "/Users/chingaling/Documents/Algo Trading/BroCodePC/stocks_to_trade.csv"
csvAddress_orders = "/Users/chingaling/Documents/Algo Trading/BroCodePC/orders.csv"
pklAdd_orders = "/Users/chingaling/Documents/Algo Trading/BroCodePC/orders.pkl"
pklAdd_exec = "/Users/chingaling/Documents/Algo Trading/BroCodePC/executions.pkl"

In [144]:
# Test connection
#clientID = 2
app = ib.TestApp(ip, port, clientID)
current_time = app.speaking_clock()
print(current_time)

Getting the time from the server... 
IB error id -1 errorcode 2110 string Connectivity between Trader Workstation and server is broken. It will be restored automatically.
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfuture
IB error id -1 errorcode 2104 string Market data farm connection is OK:eufarm
IB error id -1 errorcode 2104 string Market data farm connection is OK:njfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds.us
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ilhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:euhmds
IB error id -1 errorcode 2105 string HMDS data farm connection is broken:fundfarm
IB error id -1 errorcode 2105 string HMDS data farm connection is broken:ushmds
1542735837


## Making buy orders

In [65]:
def submit_buy_order(stock_to_trade):
    #if stock_to_trade in positions:
        #TODO: Check if this is actually a criteria in the Quantopian model
        #pass 
    resolved_ibcontract = app.create_resolved_ibcontract(stock_to_trade)
    PH = app.get_IB_historical_data(resolved_ibcontract, durationStr='20 D')
    PH_df = pd.DataFrame(PH, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    PH_Avg = float(PH_df.close.mean())
    current_price = float(app.get_latest_price(resolved_ibcontract))
    
    # If can't retrieve current price, skip it for now
    if np.isnan(current_price):
        print('Could not get current price. Skipping for now')
        pass 
    else:
        
        # If there's a large 25% surge in price, buy it at current price
        if current_price > float(1.25*PH_Avg):
            buyPrice=float(current_price)
        else:
            buyPrice=float(current_price*buyFactor)
        buyPrice=float(ib.make_div_by_05(buyPrice, buy=True))
        StockShares = int(minInvSize/buyPrice)
        print("Buying {}, \tshares: {}, \tprice: {}".format(stock_to_trade, StockShares, buyPrice))

        # Submit the buy order and log the order
        orderid = app.submit_order(resolved_ibcontract=resolved_ibcontract, action="BUY", shares=StockShares, price=buyPrice, order_type="LMT", orderid=None, tif="GTC")
        app.add_order_entry(orderid)

In [68]:
def rebalance_buy_orders():
    
    WeightThisBuyOrder=float(1.00/maxBuyOrdersAtOnce)
    
    #TODO: Update hard-coded integer here to max out my available cash
    #TODO: Update for loop to keep going until all buy orders are executed. Currently, sometimes some buy orders will be skipped
    for ThisBuyOrder in range(13):

        # Get next stock to trade
        stock_to_trade = next(app.nextStock)
        print("Considering trading this stock: {}".format(stock_to_trade))

        # TODO: Check to see if this stock is already on the order list, or within the portfolio. Or if it's been traded already today (like sold). If so, skip to the next one.

        # Check if there is an existing buy order, or if stock is already in portfolio
        positions = app.get_current_positions()
        open_orders = app.get_open_orders_pd()
        updated_executions_pd = app.get_latest_executions()
        todays_executions = updated_executions_pd[datetime.datetime.now() - updated_executions_pd.dttime < datetime.timedelta(days=1)]

        cond1 = positions.symbol.isin([stock_to_trade]).any()
        cond2 = open_orders[open_orders.action == 'BUY'].symbol.isin([stock_to_trade]).any()
        cond3 = todays_executions.symbol.isin([stock_to_trade]).any()
        if cond1 or cond2 or cond3:
            print('In positions?: {}\tIn open_orders?: {}\tTraded today?: {}'.format(cond1, cond2, cond3))
            pass
        else:
            submit_buy_order(stock_to_trade)

In [191]:
# Load the list of 100 stocks to consider trading today
app.generate_stock_shortlist()

In [192]:
# Run algorithm to create all of the buy orders.
rebalance_buy_orders()

Considering trading this stock: IMTE
Getting full contract details from the server... 
got multiple contracts using first one
284710180,IMTE,STK,,0.0,,,SMART,NASDAQ,USD,IMTE,SCM,False,,combo:
Getting historical data from the server... could take 20 seconds to complete 
IB error id 50 errorcode 366 string No historical data query found for ticker id:50
Current price: nan
Could not get current price. Skipping for now
Considering trading this stock: MBIO
Getting full contract details from the server... 
got multiple contracts using first one
286853239,MBIO,STK,,0.0,,,SMART,NASDAQ,USD,MBIO,NMS,False,,combo:
Getting historical data from the server... could take 20 seconds to complete 
IB error id 50 errorcode 366 string No historical data query found for ticker id:50
Current price: nan
Could not get current price. Skipping for now
Considering trading this stock: OPNT
Getting full contract details from the server... 
got multiple contracts using first one
225015392,OPNT,STK,,0.0,,,SMART,NASD

## Cancelling buy orders

In [201]:
# Cancel all open buy orders
app.cancel_all_open_buy_orders(minutes=rebalance_interval)

## Submitting sell orders

In [163]:
app.rebalance_all_sell_orders()

Considering placing sell order for MOTS
Getting full contract details from the server... 
got multiple contracts using first one
306363711,MOTS,STK,,0.0,,,SMART,NASDAQ,USD,MOTS,SCM,False,,combo:
Current price: 3.15
Position held for less than 2 days. Skipping
Considering placing sell order for BXG
Getting full contract details from the server... 
got multiple contracts using first one
296320248,BXG,STK,,0.0,,,SMART,NYSE,USD,BXG,BXG,False,,combo:
Current price: 11.08
Position held for less than 2 days. Skipping
Considering placing sell order for MGTA
Do not have a position
Considering placing sell order for SIC
Existing sell order in place. Skipping
Getting full contract details from the server... 
got multiple contracts using first one
329820282,SIC,STK,,0.0,,,SMART,NASDAQ,USD,SIC,SCM,False,,combo:
Current price: 8.0
Selling SIC, 	shares: 232.0, 	price: 8.7
Using order id of 49
Placed order, orderid is 49
Considering placing sell order for LEVB
Getting full contract details from the se

In [ ]:
def main_trading():
    # Here goes all of the code to run throughout the trading day

    #Before trading day starts, load up list of stock candidates
    self.stock_list = generate_stock_shortlist()

    #TODO: Every rebalance_interval minutes, rebalance the portfolio
    while (current_time < end_trading_day):
        
        # Connect to the IB API for every rebalancing
        app = ib.TestApp("127.0.0.1", port, 1)
        rebalance(buyFactor=buyFactor, SellFactor=SellFactor)
        
        # At the end of the day, disconnect from the IB API app
        app.disconnect()
        
        # Wait for the rebalance_interval time (in minutes)
        time.wait(rebalance_interval * 60)

In [ ]:
def rebalance(buyFactor=.99, SellFactor=1.01):
    # Re-balancing method from 30_stock algo every 10 minutes. Buy order cycles through list of 100 stock selected each day.

    # Get outstanding buying power from account
    cash=get_buying_power()

    # Close out all outstanding buy orders that were not fulfilled. Should be most of them
    cancel_open_buy_orders()

    # Get list of current positions
    positions = get_current_positions()

    # Create new sell orders as per algo strategy
    rebalance_all_sell_orders()

    # Submit new buy orders
    if cash > minInvSize:
        rebalance_buy_orders()

In [ ]:
# Retrieve information from IB server
def get_buying_power(self):
    #TODO: Retrieve remaining buying power from account
    accounting_values = app.get_accounting_values(accountName)
    
    pass

In [ ]:
app.cancel_open_buy_orders()

In [ ]:
def get_open_sell_orders(self):
    #TODO: Retrieve list of all open sell orders:

    pass

In [ ]:
def generate_stock_shortlist(self):

    # Return list of stocks to trade for the day
    csvAddress = "/Users/chingaling/Documents/Algo Trading/BroCodePC/stocks_to_trade.csv"
    self.stock_list = list(pd.read_csv(csvAddress).Stocks)
    self.nextStock = cycle(stockNames)

In [ ]:
app.generate_stock_shortlist()

## Getting executions and commission

In [ ]:
## lets get positions
positions_list = app.get_current_positions()
print(positions_list)

## get the account name from the position
## normally you would know your account name
accountName = positions_list.account[0]

## and accounting information
accounting_values = app.get_accounting_values(accountName)
print(accounting_values)

## these values are cached
## if we ask again in more than 5 minutes it will update everything
accounting_updates = app.get_accounting_updates(accountName)
print(accounting_updates)

# TODO: solve this error below
'''
  File "/Users/chingaling/Documents/Algo Trading/BroCodePC/IB_API_tutorial.py", line 380, in updateAccountTime
    self._my_accounts[accountName].put(data)
NameError: name 'accountName' is not defined
'''

In [ ]:
accounting_values

In [ ]:
accounting_updates[0]

In [ ]:
# Re-balancing method from 30_stock algo every 10 minutes. Buy order cycles through list of 100 stock selected each day.

def my_rebalance(context, data):
    buyFactor=.99
    SellFactor=1.01
    cash=context.portfolio.cash

    cancel_open_buy_orders(context, data)

    # Order sell at profit target in hope that somebody actually buys it
    for stock in context.portfolio.positions:
        if not get_open_orders(stock):
            StockShares = context.portfolio.positions[stock].amount
            CurrPrice = float(data.current([stock], 'price'))
            CostBasis = float(context.portfolio.positions[stock].cost_basis)
            SellPrice = float(make_div_by_05(CostBasis*SellFactor, buy=False))
            
            
            if np.isnan(SellPrice) :
                pass # probably best to wait until nan goes away
            elif (stock in context.age and context.age[stock] == 1) :
                pass
            elif (
                stock in context.age 
                and context.MyFireSaleAge<=context.age[stock] 
                and (
                    context.MyFireSalePrice>CurrPrice
                    or CostBasis>CurrPrice
                )
            ):
                if (stock in context.age and context.age[stock] < 2) :
                    pass
                elif stock not in context.age:
                    context.age[stock] = 1
                else:
                    SellPrice = float(make_div_by_05(.95*CurrPrice, buy=False))
                    print("Selling {}, \tshares: {}, \tprice: {}".format(stock, StockShares, SellPrice))
                    order(stock, -StockShares,
                        style=LimitOrder(SellPrice)
                    )
            else:
                if (stock in context.age and context.age[stock] < 2) :
                    pass
                elif stock not in context.age:
                    context.age[stock] = 1
                else:
                    print("Selling {}, \tshares: {}, \tprice: {}".format(stock, StockShares, SellPrice))
                    order(stock, -StockShares,
                        style=LimitOrder(SellPrice)
                    )

    WeightThisBuyOrder=float(1.00/context.maxBuyOrdersAtOnce)
    for ThisBuyOrder in range(context.maxBuyOrdersAtOnce):
        stock = next(context.MyCandidate)
        PH = data.history([stock], 'price', 20, '1d')
        PH_Avg = float(PH.mean())
        CurrPrice = float(data.current([stock], 'price'))
        if np.isnan(CurrPrice):
            pass # probably best to wait until nan goes away
        else:
            if CurrPrice > float(1.25*PH_Avg):
                BuyPrice=float(CurrPrice)
            else:
                BuyPrice=float(CurrPrice*buyFactor)
            BuyPrice=float(make_div_by_05(BuyPrice, buy=True))
            StockShares = int(WeightThisBuyOrder*cash/BuyPrice)
            #print("Buying {}, \tshares: {}, \tprice: {}".format(stock, StockShares, BuyPrice))
            order(stock, StockShares,
                style=LimitOrder(BuyPrice)
            )

## Lesson 1: Create a connection with IB API
https://qoppac.blogspot.com/2017/03/interactive-brokers-native-python-api.html

## Lesson 2: Get historic data
https://qoppac.blogspot.com/2017/03/historic-data-from-native-ib-pyhon-api.html

## Lesson 3: Get streaming data
https://qoppac.blogspot.com/2017/03/streaming-market-data-from-native.html

## Lesson 4: Creating orders
https://qoppac.blogspot.com/2017/03/placing-orders-in-native-python-ib-api.html

## Lesson 5: Getting portfolio positions and account info
https://qoppac.blogspot.com/2017/03/getting-position-and-accounting-data.html